In [ ]:
# Using Selenium to run NOAA VERTCON 

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4  import BeautifulSoup as bs # Scraping webpages
import re
import csv
import os
import pandas as pd

def getdatumshift(lat_, lon_, H_, csv_name):
    PATH = "C:\Program Files (x86)\chromedriver.exe"
    driver = webdriver.Chrome(PATH)
    
    driver.get("https://geodesy.noaa.gov/cgi-bin/VERTCON/vert_con.prl")
    print(driver.title)

    search = driver.find_element_by_name("LAT")
    search.send_keys(lat_)
    search = driver.find_element_by_name("LON")
    search.send_keys(lon_)
    search = driver.find_element_by_name("HGT")
    search.send_keys(H_)
    search.send_keys(Keys.RETURN)
    
    time.sleep(2)
    pg_source_txt = driver.page_source
    soup = bs(pg_source_txt, "html")

    lat_tag = (soup.body.findAll(text = re.compile("Latitude*"))[0])
    print (lat_tag)
    res_lat = float(lat_tag.split(":")[1])
    print(res_lat)

    lon_tag = (soup.body.findAll(text = re.compile("Longitude*"))[0])
    print (lon_tag)
    res_lon = float(lon_tag.split(":")[1]) 
    print(res_lon)
    
    ngvd29_tag = (soup.body.findAll(text = re.compile("NGVD 29 height*"))[0])
    print (ngvd29_tag)
    ngvd29 = float((ngvd29_tag.split(":")[1]).split("FT")[0])
    print (ngvd29)
    #print((ngvd29_tag.split(":")[1]))
    
    dt_shft_tag = (soup.body.findAll(text = re.compile("Converted to NAVD 88 height*"))[0])
    print (dt_shft_tag)
    dt_shft = float((dt_shft_tag.split(":")[1]).split("feet")[0])
    print (dt_shft)
    
    res = [res_lat, res_lon, ngvd29, dt_shft]
    driver.close()
    return (res)



#lat_ = [40.364307, 40.173636]
#lon_ = [111.887446, 111.888599]
#H_ = [4494.300, 4494.500] 

df = pd.read_csv(r"C:\Users\Sulochan Dhungel\Desktop\junk\Book3.csv")
lat_ = df['Lat_'].tolist()[0:2510:5]
lon_ = df['Lon_'].tolist()[0:2510:5]
lon_ = [x*-1 for x in lon_]
H_ = df['Elev'].tolist()[0:2510:5]


csv_name = r"C:\Users\Sulochan Dhungel\Desktop\junk\ngvd_to_navd88.csv"
if os.path.exists(csv_name):
    csv_created = True
else:
    csv_created = False

for i in range(0,len(lat_)):
    lat_str = str(lat_[i])
    lon_str = str(lon_[i])
    H_str = str(format(H_[i], '.4f')) + " FT"
    
    res = getdatumshift(lat_str, lon_str, H_str, csv_name)
    print (res)
    if csv_created:
        with open(csv_name, 'a', newline="") as csv_obj:
            writer_obj = csv.writer(csv_obj)
            writer_obj.writerow(res)
            csv_obj.close()
    else:
        with open(csv_name, 'w') as csv_obj:
            writer_obj = csv.writer(csv_obj)
            writer_obj.writerow(["Lat", "Lon (W)", "NGVD29", "NAVD88"])
            writer_obj.writerow(res)
            csv_obj.close()
            csv_created = True


    
    

